In [ ]:
import Pkg
pkgs = [
"DataFrames",
"Dates",
"uCSV",
"Revise",
]
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end
import Mycelia

In [ ]:
# mamba create -n mmseqs2 -c bioconda mmseqs2

# conda run --no-capture-output -n mmseqs2 mmseqs 

In [ ]:
# MMSeqs Easy taxonomy
# amino acid
# ~ 5 hour download time may not be tolerable?
# processing
# 7930.350543 seconds (710.71 k allocations: 40.309 MiB, 0.00% gc time, 0.00% compilation time)
# 2 hours 12m
# 1.5 hours for iterative search with varying sensitivity
# Mycelia.download_mmseqs_db(db="UniRef100", conda_env="viral-pangenome-discovery")
# --force-reuse 1
# --remove-tmp-files 1
# conda run --live-stream -n viral-pangenome-discovery

# download on NERSC is way too slow - going to skip for now but will consider retrying later
# mkdir -p $HOME/workspace/mmseqs
# conda run --no-capture-output -n mmseqs2 mmseqs databases --compressed 1 UniRef100 $HOME/workspace/mmseqs/UniRef100 $HOME/workspace/mmseqs/tmp
# conda run --no-capture-output -n mmseqs2 mmseqs databases --compressed 1 UniRef90 $HOME/workspace/mmseqs/UniRef90 $HOME/workspace/mmseqs/tmp
# conda run --no-capture-output -n mmseqs2 mmseqs databases --compressed 1 UniRef50 $HOME/workspace/mmseqs/UniRef50 $HOME/workspace/mmseqs/tmp

In [ ]:
data_dir = joinpath(dirname(pwd()), "data")

In [ ]:
SRR_paths = filter(x -> !occursin(".ipynb_checkpoints", x), readdir(joinpath(data_dir, "SRA"), join=true))
SRR_paths = filter(x -> isfile(joinpath(x, "megahit", "final.contigs.fastg.gfa.fna")), SRR_paths)
# filter out samples that have already been processed
# SRR_paths = filter(x -> !isfile(joinpath(x, basename(x) * ".contig_info.tsv")), SRR_paths)

In [ ]:
# load in metadata
metadata_dir = joinpath(dirname(pwd()), "metadata")

exposome_environmental_data = DataFrames.DataFrame(uCSV.read(
    joinpath(metadata_dir, "metadata_exposome.rds.tsv"),
    delim='\t',
    header=1,
    typedetectrows=300
))

joint_sample_metadata = DataFrames.DataFrame(uCSV.read(
    joinpath(metadata_dir, "exposome/joint_sample_metadata.tsv"),
    delim='\t',
    header=1,
    typedetectrows=300
))

@assert joint_sample_metadata[!, "Library Name"] == joint_sample_metadata[!, "LibraryName"]

joint_metadata = DataFrames.innerjoin(
    joint_sample_metadata,
    exposome_environmental_data,
    on="Library Name" => "samplenames")

run_ids = sort(joint_metadata[!, "Run"])

sample_paths = joinpath.(data_dir, "SRA", run_ids)

In [ ]:
target_database = "$(homedir())/workspace/mmseqs/UniRef100"
for sample_path in sample_paths[2:end]
    SRR = basename(sample_path)
    assembled_fasta = joinpath(sample_path, "megahit", "final.contigs.fastg.gfa.fna")
    out_dir = mkpath(joinpath(sample_path, "mmseqs_easy_search"))
    outfile_base = joinpath(out_dir, basename(assembled_fasta) * ".mmseqs_easy_search." * basename(target_database))
    run(`sbatch $(homedir())/workspace/Mycelia/scripts/mmseqs-easy-search.sh $assembled_fasta $target_database $outfile_base`)
    # println("conda run --no-capture-output -n mmseqs2 mmseqs easy-taxonomy $(assembled_fasta) $(target_database) $(outfile_base) $(joinpath(out_dir, "tmp"))")
    # Mycelia.run_mmseqs_easy_search(out_dir=SRR_path, query_fasta=assembled_fasta, target_database=target_database, outfile=SRR)
end